In [1]:
import torch
import sys
import os
import lightning as L
from lightning.pytorch.callbacks import ModelCheckpoint, EarlyStopping
from lightning.pytorch.loggers import MLFlowLogger

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))


if project_root not in sys.path:
    sys.path.append(project_root)
    
from model_utils import ModelKANLinear, ClassificationData, no_augmentation

torch.cuda.empty_cache()
torch.set_float32_matmul_precision("medium")
L.seed_everything(111)

Seed set to 111


111

### Data Augmentation

In [2]:
example_hyperparams = {
    "learning_rate": 0.1,
    "batch_size": 512,
    "weight_decay": 0.001,
    "dropout": 0.2,
}
model = ModelKANLinear(batch_size=example_hyperparams["batch_size"], learning_rate=example_hyperparams["learning_rate"], weight_decay=example_hyperparams["weight_decay"], dropout=example_hyperparams["dropout"])
logger = MLFlowLogger(save_dir="mlruns", experiment_name="KAN-Linear")
early_stop = EarlyStopping(monitor="train_loss", patience=3, mode="min", verbose=True, min_delta=0.01)
checkpoint_callback = ModelCheckpoint(monitor="val_f1_macro", mode="max", dirpath="checkpoints/kan-linear", filename=f"lr={model.hyperparameters["learning_rate"]}_bs={model.hyperparameters["batch_size"]}_wd={model.hyperparameters["weight_decay"]}_dropout={model.hyperparameters["dropout"]}" + "-{epoch:02d}-{val_f1_macro:.2f}")
trainer = L.Trainer(max_epochs=20, logger=logger, num_sanity_val_steps=0, enable_model_summary=False, deterministic=False, callbacks=[early_stop, checkpoint_callback], precision="16-mixed")
data = ClassificationData(batch_size=model.hyperparameters["batch_size"])
trainer.fit(model, datamodule=data)
trainer.test(model, datamodule=data)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 0: 100%|██████████| 176/176 [00:33<00:00,  5.22it/s, v_num=dcea]

Metric train_loss improved. New best score: 1.645


Epoch 1: 100%|██████████| 176/176 [00:17<00:00, 10.19it/s, v_num=dcea]

Metric train_loss improved by 0.195 >= min_delta = 0.01. New best score: 1.451


...

`Trainer.fit` stopped: `max_epochs=20` reached.


Epoch 19: 100%|██████████| 176/176 [00:16<00:00, 10.52it/s, v_num=dcea]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\SPCX\Desktop\github-repositories\dl-cnn\.venv\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:420: Consider setting `persistent_workers=True` in 'test_dataloader' to speed up the dataloader worker initialization.


Testing DataLoader 0: 100%|██████████| 176/176 [00:07<00:00, 22.75it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       test_auroc           0.9162010550498962
      test_f1_macro          0.567167341709137
     test_precision         0.5699481964111328
       test_recall          0.5689666867256165
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_auroc': 0.9162010550498962,
  'test_f1_macro': 0.567167341709137,
  'test_precision': 0.5699481964111328,
  'test_recall': 0.5689666867256165}]

### No Data Augmentation

In [4]:
model = ModelKANLinear(batch_size=example_hyperparams["batch_size"], learning_rate=example_hyperparams["learning_rate"], weight_decay=example_hyperparams["weight_decay"], dropout=example_hyperparams["dropout"])
logger = MLFlowLogger(save_dir="mlruns", experiment_name="KAN-Linear-no-augmentation")
early_stop = EarlyStopping(monitor="train_loss", patience=3, mode="min", verbose=True, min_delta=0.01)
checkpoint_callback = ModelCheckpoint(monitor="val_f1_macro", mode="max", dirpath="checkpoints/kan-linear", filename=f"no-augment_lr={model.hyperparameters["learning_rate"]}_bs={model.hyperparameters["batch_size"]}_wd={model.hyperparameters["weight_decay"]}_dropout={model.hyperparameters["dropout"]}" + "-{epoch:02d}-{val_f1_macro:.2f}")
trainer = L.Trainer(max_epochs=20, logger=logger, num_sanity_val_steps=0, enable_model_summary=False, deterministic=False, callbacks=[early_stop, checkpoint_callback], precision="16-mixed")
data = ClassificationData(batch_size=model.hyperparameters["batch_size"], transform=no_augmentation)
trainer.fit(model, datamodule=data)
trainer.test(model, datamodule=data)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 0: 100%|██████████| 176/176 [00:28<00:00,  6.12it/s, v_num=138b]

Metric train_loss improved. New best score: 1.427


Epoch 1: 100%|██████████| 176/176 [00:13<00:00, 13.47it/s, v_num=138b]

Metric train_loss improved by 0.256 >= min_delta = 0.01. New best score: 1.171


...

Metric train_loss improved by 0.016 >= min_delta = 0.01. New best score: 0.336
`Trainer.fit` stopped: `max_epochs=20` reached.


Epoch 19: 100%|██████████| 176/176 [00:12<00:00, 14.10it/s, v_num=138b]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 176/176 [00:03<00:00, 46.82it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       test_auroc            0.909548282623291
      test_f1_macro         0.5702996850013733
     test_precision         0.5794668197631836
       test_recall          0.5667333602905273
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_auroc': 0.909548282623291,
  'test_f1_macro': 0.5702996850013733,
  'test_precision': 0.5794668197631836,
  'test_recall': 0.5667333602905273}]